In [1]:
import pandas as pd
import glob
import os

In [2]:
# Definir o caminho da pasta onde os arquivos CSV estão localizados
caminho = os.path.join("files", "csv_2")

In [3]:
# Função para obter todos os arquivos CSV na pasta
def obter_arquivos_csv(caminho):
    """Retorna uma lista de caminhos dos arquivos CSV no diretório especificado."""
    arquivos = glob.glob(os.path.join(caminho, '*.csv'))
    if not arquivos:
        print("Nenhum arquivo CSV encontrado.")
    return arquivos

In [4]:
# Função para ler e processar os arquivos CSV
def ler_arquivos_csv(arquivos_csv):
    """Lê todos os arquivos CSV e retorna uma lista de DataFrames."""
    lista = []
    for arquivo in arquivos_csv:
        try:
            df = pd.read_csv(arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])
            lista.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    return lista

In [5]:
# Função para concatenar os DataFrames
def concatenar_dataframes(lista):
    """Concatena todos os DataFrames em um único DataFrame."""
    if not lista:
        print('Nenhum dado para concatenar.')
        return None
    return pd.concat(lista, ignore_index=True)

In [ ]:
# Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
def adicionar_nome_microinversor(df):
    """
    Adiciona uma coluna 'Microinversor' baseada na coluna 'SN'.
    Associa cada SN a um nome de microinversor único.
    """
    if 'SN' not in df.columns and 'Sn' in df.columns:
        df.rename(columns={'Sn': 'SN'}, inplace=True)

    if 'SN' not in df.columns:
        print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
        return df

    # Criar um dicionário associando os SNs a nomes de microinversores
    microinversores_dict = {sn: f"Microinversor_{idx+1}" for idx, sn in enumerate(df['SN'].unique())}

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)

    return df

In [ ]:
# Função para substituir dados antigos pela mesma data e microinversor
def obter_dataframe_final(caminho_novo_arquivo, df_novo):
    """Remove do arquivo existente os dados da mesma data e microinversor antes de salvar os novos."""
    if os.path.exists(caminho_novo_arquivo):
        try:
            df_existente = pd.read_csv(caminho_novo_arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])

            # Se 'Microinversor' não existir ainda, criamos a coluna antes da remoção
            if 'Microinversor' not in df_existente.columns:
                df_existente = adicionar_nome_microinversor(df_existente)

            # Garantir que df_novo também tem a coluna Microinversor
            df_novo = adicionar_nome_microinversor(df_novo)

            # Remover do dataset existente qualquer entrada que tenha a mesma 'Date' e 'Microinversor'
            df_existente = df_existente[~df_existente[['Date', 'Microinversor']].apply(tuple, axis=1).isin(
                df_novo[['Date', 'Microinversor']].apply(tuple, axis=1))]

            # Concatenar os dados garantindo que os mais recentes substituam os antigos
            df_final = pd.concat([df_existente, df_novo]).reset_index(drop=True)

            return df_final
        except Exception as e:
            print(f"Erro ao ler o arquivo existente: {e}")
            return df_novo  # Se der erro, mantém apenas os novos dados
    return df_novo

In [ ]:
# Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
def adicionar_nome_microinversor(df):
    """
    Adiciona uma coluna 'Microinversor' baseada na coluna 'SN'.
    Associa cada SN a um nome de microinversor único.
    """
    if 'SN' not in df.columns and 'Sn' in df.columns:
        df.rename(columns={'Sn': 'SN'}, inplace=True)

    if 'SN' not in df.columns:
        print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
        return df

    # Criar um dicionário associando os SNs a nomes de microinversores
    microinversores_dict = {sn: f"Microinversor_{idx+1}" for idx, sn in enumerate(df['SN'].unique())}

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)

    return df

In [7]:
# Função para salvar o DataFrame atualizado
def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Função principal
def main():
    caminho_novo_arquivo = os.path.join(caminho, 'arquivo.csv')

    arquivos_csv = obter_arquivos_csv(caminho)
    if not arquivos_csv:
        return  # Encerra o script se não houver arquivos

    lista = ler_arquivos_csv(arquivos_csv)
    df_novo = concatenar_dataframes(lista)

    if df_novo is not None:
        df_novo = adicionar_nome_microinversor(df_novo)  # Adiciona a coluna do microinversor com base no SN
        # df_final = obter_dataframe_final(caminho_novo_arquivo, df_novo)
        display(df_novo)
        # salvar_dataframe(df_novo, caminho_novo_arquivo)

In [8]:
if __name__ == "__main__":
    main()

,Plant Name,Date,SN,Port,Energy (kWh),Microinversor
0,Wilkne,2025-01-02,106272403152,1,2.061,Microinversor_1
1,Wilkne,2025-01-02,106272403152,2,2.066,Microinversor_1
2,Wilkne,2025-01-02,106272403152,3,2.065,Microinversor_1
3,Wilkne,2025-01-02,106272403152,4,2.087,Microinversor_1
4,Wilkne,2025-01-02,106272403916,1,2.078,Microinversor_2
5,Wilkne,2025-01-02,106272403916,2,2.073,Microinversor_2
6,Wilkne,2025-01-02,106272403916,3,2.047,Microinversor_2
7,Wilkne,2025-01-02,106272403916,4,2.091,Microinversor_2
8,Wilkne,2025-01-02,106272404964,1,0.012,Microinversor_3
9,Wilkne,2025-01-02,106272404964,2,1.848,Microinversor_3
